In [1]:
from scipy.io import loadmat
import pandas as pd
import numpy as np

In [2]:
!wget https://monujohn.com/EEGdata/WLDataAll.mat

--2021-08-04 07:57:33--  https://monujohn.com/EEGdata/WLDataAll.mat
Resolving monujohn.com (monujohn.com)... 198.136.51.114
Connecting to monujohn.com (monujohn.com)|198.136.51.114|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42375907 (40M)
Saving to: ‘WLDataAll.mat’

WLDataAll.mat       100%[===================>]  40.41M  29.8MB/s    in 1.4s    

2021-08-04 07:57:35 (29.8 MB/s) - ‘WLDataAll.mat’ saved [42375907/42375907]



In [3]:
annots = loadmat('WLDataAll.mat')

In [4]:
data = annots['data']

In [5]:
datapoint_array = []

def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

for datapoint in data:
  datapoint = datapoint.T
  data_squared = np.square(datapoint)
  data_sum = data_squared.sum(axis=1)
  
  data_normalized = NormalizeData(data_sum)
  datapoint_array.append(data_normalized)


datapoint_array = np.array(datapoint_array)

datapoint_array = datapoint_array.transpose()

print('Latest array shape:', datapoint_array.shape)

Latest array shape: (360, 62)


In [6]:
X = datapoint_array
Y = annots['label'].transpose()

Y = np.where(Y == 1, 0, Y)
Y = np.where(Y == 2, 1, Y)

print('X shape:', X.shape)
print('Y shape:', Y.shape)

X shape: (360, 62)
Y shape: (360, 1)


In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

In [8]:
import tensorflow as tf
from tensorflow import keras

model = keras.Sequential([
    keras.layers.Flatten(input_shape=(62,)),
    keras.layers.Dense(16, activation=tf.nn.relu),
	keras.layers.Dense(16, activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid),
])

In [9]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=100, batch_size=1)
test_loss, test_acc = model.evaluate(X_test, y_test)

Epoch 1/100
252/252 [==============================] - 1s 1ms/step - loss: 0.6990 - accuracy: 0.4444
Epoch 2/100
252/252 [==============================] - 0s 1ms/step - loss: 0.6947 - accuracy: 0.5119
Epoch 3/100
252/252 [==============================] - 0s 1ms/step - loss: 0.6952 - accuracy: 0.5000
Epoch 4/100
252/252 [==============================] - 0s 1ms/step - loss: 0.6955 - accuracy: 0.5198
Epoch 5/100
252/252 [==============================] - 0s 1ms/step - loss: 0.6937 - accuracy: 0.4960
Epoch 6/100
252/252 [==============================] - 0s 1ms/step - loss: 0.6913 - accuracy: 0.5278
Epoch 7/100
252/252 [==============================] - 0s 1ms/step - loss: 0.6911 - accuracy: 0.5357
Epoch 8/100
252/252 [==============================] - 0s 1ms/step - loss: 0.6891 - accuracy: 0.5238
Epoch 9/100
252/252 [==============================] - 0s 1ms/step - loss: 0.6843 - accuracy: 0.5595
Epoch 10/100
252/252 [==============================] - 0s 1ms/step - loss: 0.6783 - accura

In [10]:
print('Test accuracy:', test_acc)

Test accuracy: 0.6759259104728699
